# Where does China rank in population growth, birth rate? 

Population growth rate ([data download here](https://www.cia.gov/the-world-factbook/field/population-growth-rate/country-comparison)) compares the average annual percent change in populations, resulting from a surplus (or deficit) of births over deaths and the balance of migrants entering and leaving a country. The rate may be positive or negative.

Birth rate ([download data here](https://www.cia.gov/the-world-factbook/field/birth-rate/country-comparison)) compares the average annual number of births during a year per 1,000 persons in the population at midyear; also known as crude birth rate.

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
from datetime import timedelta
import numpy as np

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

### Grab specific tables from [CIA World Factbook](https://www.cia.gov/the-world-factbook/)

In [4]:
df_growth = pd.read_csv(
    "data/raw/export.csv",
    header=0,
    names=["country", "cia-slug", "growth_rate", "date", "rank", "region"],
)

In [5]:
df_birth = pd.read_csv(
    "data/raw/birth_rate_cia.csv",
    header=0,
    names=["country", "cia-slug", "birth_rate", "date", "rank", "region"],
)

---

### OECD countries

In [6]:
oecd_src = pd.read_html(
    "https://www.oecd.org/about/document/ratification-oecd-convention.htm", header=0
)[1]

In [7]:
oecd_src.drop(["Unnamed: 0", "Unnamed: 3"], axis=1, inplace=True)

In [8]:
oecd_src.columns = oecd_src.columns.str.lower()
oecd_src["country"] = oecd_src["country"].str.title()
oecd_src["date"] = pd.to_datetime(oecd_src["date"])

In [9]:
oecd_list = oecd_src.country.to_list()

---

### Largest economies

In [10]:
econ_list = [
    "United States",
    "China",
    "Japan",
    "Germany",
    "United Kingdom",
    "India",
    "France",
    "Italy",
    "Canada",
    "South Korea",
]

---

### In OECD?

In [11]:
df_growth["oecd"] = df_growth["country"].isin(oecd_list)

In [12]:
df_birth["oecd"] = df_birth["country"].isin(oecd_list)

### Top 10 economies?

In [13]:
df_growth["econ_top"] = df_growth["country"].isin(econ_list)

In [14]:
df_birth["econ_top"] = df_birth["country"].isin(econ_list)

---

### Growth rate

In [15]:
df_growth.head()

,country,cia-slug,growth_rate,date,rank,region,oecd,econ_top
0,South Sudan,south-sudan,5.05,2021 est.,1,Africa,False,False
1,Burundi,burundi,3.68,2021 est.,2,Africa,False,False
2,Niger,niger,3.65,2021 est.,3,Africa,False,False
3,Angola,angola,3.38,2021 est.,4,Africa,False,False
4,Benin,benin,3.36,2021 est.,5,Africa,False,False


In [16]:
asian_nations_growth = df_growth[df_growth["country"] == "East Asia/Southeast Asia"]

In [17]:
top_econ_growth = df_growth[df_growth["econ_top"] == True]

In [18]:
df_growth.head()

,country,cia-slug,growth_rate,date,rank,region,oecd,econ_top
0,South Sudan,south-sudan,5.05,2021 est.,1,Africa,False,False
1,Burundi,burundi,3.68,2021 est.,2,Africa,False,False
2,Niger,niger,3.65,2021 est.,3,Africa,False,False
3,Angola,angola,3.38,2021 est.,4,Africa,False,False
4,Benin,benin,3.36,2021 est.,5,Africa,False,False


In [19]:
df_growth.groupby(["region"]).agg({"growth_rate": "mean"}).reset_index().sort_values(
    "growth_rate", ascending=False
)

,region,growth_rate
0,Africa,2.171636
6,Middle East,1.128333
9,South Asia,1.078750
8,South America,0.893846
4,East Asia/Southeast Asia,0.848095
3,Central Asia,0.805000
2,Central America,0.707187
1,Australia - Oceania,0.352692
7,North America,0.276667
5,Europe,0.135600


In [20]:
df_growth.head()

,country,cia-slug,growth_rate,date,rank,region,oecd,econ_top
0,South Sudan,south-sudan,5.05,2021 est.,1,Africa,False,False
1,Burundi,burundi,3.68,2021 est.,2,Africa,False,False
2,Niger,niger,3.65,2021 est.,3,Africa,False,False
3,Angola,angola,3.38,2021 est.,4,Africa,False,False
4,Benin,benin,3.36,2021 est.,5,Africa,False,False


---

### Birth rate

In [21]:
df_birth.head()

,country,cia-slug,birth_rate,date,rank,region,oecd,econ_top
0,Niger,niger,47.28,2021 est.,1,Africa,False,False
1,Angola,angola,42.22,2021 est.,2,Africa,False,False
2,Mali,mali,41.60,2021 est.,3,Africa,False,False
3,Uganda,uganda,41.60,2021 est.,4,Africa,False,False
4,Benin,benin,41.55,2021 est.,5,Africa,False,False


In [22]:
asian_nations_birth = df_birth[df_birth["country"] == "East Asia/Southeast Asia"]

In [23]:
top_econ_birth = df_birth[df_birth["econ_top"] == True]

### Exports

In [24]:
df_growth.to_csv("data/processed/china_other_countries_growth.csv", index=False)